# Linear Regression with MLFLow

Train a linear regression model that takes two hyperparameters: alpha and l1_ratio.

In [1]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# Modeling wine preferences by data mining from physicochemical properties.

#Import Libraries
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
from mlflow.models.signature import infer_signature
import mlflow.sklearn
import logging
import matplotlib.pyplot as plt

In [2]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


#Helper function for evaluation metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [3]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    #alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.5
    #l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.5
    #I'm getting errors trying to convert the strings to float, so I'm hard coding the hyper parameters and varying them manually between runs
    alpha = 0
    l1_ratio = 0

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        #Train model
        lr.fit(train_x, train_y)
        
        #make predictions on test set
        predicted_qualities = lr.predict(test_x)
        
        #get the evaluation metrics
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        #Print results
        print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        #Log parameters and metrics
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        #make predictions on train set
        predictions = lr.predict(train_x)
        signature = infer_signature(train_x, predictions)

        #register url
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(
                lr, "model", registered_model_name="ElasticnetWineModel", signature=signature
            )
        else:
            mlflow.sklearn.log_model(lr, "model", signature=signature)

Elasticnet model (alpha=0.000000, l1_ratio=0.000000):
  RMSE: 0.660381122446162
  MAE: 0.5073644199888925
  R2: 0.38209312191101596


In [ ]:
    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="Wine_Predictions",

In [7]:
#Look at MLFlow metric, parameters and artifacts in UI
!mlflow ui
#!mlflow ui --host 0.0.0.0

[2023-06-08 10:49:42 -0500] [69967] [INFO] Starting gunicorn 20.1.0
[2023-06-08 10:49:42 -0500] [69967] [INFO] Listening at: http://127.0.0.1:5000 (69967)
[2023-06-08 10:49:42 -0500] [69967] [INFO] Using worker: sync
[2023-06-08 10:49:42 -0500] [69968] [INFO] Booting worker with pid: 69968
[2023-06-08 10:49:42 -0500] [69969] [INFO] Booting worker with pid: 69969
[2023-06-08 10:49:42 -0500] [69970] [INFO] Booting worker with pid: 69970
[2023-06-08 10:49:42 -0500] [69971] [INFO] Booting worker with pid: 69971
^C
[2023-06-08 10:49:52 -0500] [69967] [INFO] Handling signal: int
[2023-06-08 10:49:52 -0500] [69969] [INFO] Worker exiting (pid: 69969)
[2023-06-08 10:49:52 -0500] [69968] [INFO] Worker exiting (pid: 69968)
[2023-06-08 10:49:52 -0500] [69970] [INFO] Worker exiting (pid: 69970)
[2023-06-08 10:49:52 -0500] [69971] [INFO] Worker exiting (pid: 69971)


Let's Examine the data set we used for our linear model

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
len(data)

In [ ]:
data.dtypes

In [ ]:
# Plot histograms for each variable in the dataframe
data.hist(bins='auto',figsize=(10, 6))

# Adjust spacing between subplots
plt.tight_layout()

# Display the histograms
plt.show()